<a href="https://colab.research.google.com/github/Harshkotkar/AI-agents-/blob/main/Webbase_loader_(Rag).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U langchain langchain_community langchain_core
!pip install langchain-text-splitter
!pip install faiss-cpu
!pip install pypdf
!pip install bs4
!pip install langchain_google_genai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 117.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
ERROR: Could not find a version that satisfies the requirement langchain-text-splitter (from versions: none)
ERROR: No matching distribution found for langchain-text-splitter
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 78.2 MB/s eta 0:00:00

In [ ]:
!pip install langchain_chroma

In [ ]:
import getpass
import os
os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API key: ")




In [ ]:
# documetn datastructure what exactly is it ?
"""The Document data structure in Langchain is a container for text data and its associated metadata.
 It has two main parts: page_content which holds the text itself, and metadata which is a dictionary for storing extra information like the source or page number.
  This helps Langchain organize and understand the text it's working with."""



In [ ]:
import bs4
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

In [ ]:
llm=ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp",temperature=0)


In [ ]:
loder=WebBaseLoader(web_path="https://www.hri.org.au/health/learn/your-body/the-human-heart",
                    bs_kwargs=dict(
                        parse_only=bs4.SoupStrainer(id="main-content")
                    ))

In [ ]:
docs=loder.load()

In [ ]:
text_split=RecursiveCharacterTextSplitter(chunk_size=1000,
                                          chunk_overlap=200)
split=text_split.split_documents(docs)

In [ ]:
split[:3]

In [ ]:
model_name="BAAI/bge-Base-en-v1.5"
model=HuggingFaceEmbeddings(model_name=model_name,
                                model_kwargs={"device":"cpu"},
                                encode_kwargs={"normalize_embeddings":True})

In [ ]:
db=Chroma.from_documents(documents=split,
                         embedding=model)

In [ ]:
retriver=db.as_retriever() # its work is to find the most relevant parts of your documents that relate to your question

In [ ]:
prompt = PromptTemplate.from_template("""Answer  the question based only on the provided context answer as you are answering a childern of "12" years:

<context>
{context}
</context>

Question: {input}""")

In [ ]:
def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
chain=({"context":retriver |format_docs ,"input": RunnablePassthrough()} # simply takes the input it receives and passes it along without changing it. It's useful for when you want to include the original input in a chain of operations
       | prompt
       | llm
       | StrOutputParser())

In [ ]:
query="What is the heart?"
result=chain.invoke(query)
print(result)